<a href="https://colab.research.google.com/github/Erfan-Mostafiz/CSE499_Emotion-Analysis/blob/Erfan/CSE499_Emotion_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/MyDrive/CSE499_EmotionAnalysis/DeepVANet-main.zip"

Archive:  /content/gdrive/MyDrive/CSE499_EmotionAnalysis/DeepVANet-main.zip
   creating: DeepVANet-main/
  inflating: DeepVANet-main/dataset.py  
  inflating: DeepVANet-main/data_preprocess.py  
  inflating: DeepVANet-main/decision_level_fusion.py  
  inflating: DeepVANet-main/demo.py  
  inflating: DeepVANet-main/models.py  
  inflating: DeepVANet-main/pretrained_cnn.pth  
  inflating: DeepVANet-main/readme.md  
  inflating: DeepVANet-main/train.py  
  inflating: DeepVANet-main/utils.py  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/gdrive/MyDrive/CSE499_EmotionAnalysis/Reduced/datasets.zip"

Archive:  /content/gdrive/MyDrive/CSE499_EmotionAnalysis/Reduced/datasets.zip
   creating: datasets/
   creating: datasets/DEAP/
   creating: datasets/DEAP/data_preprocessed_python/
   creating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/
  inflating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/s01.dat  
  inflating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/s02.dat  
  inflating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/s03.dat  
  inflating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/s04.dat  
  inflating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/s05.dat  
  inflating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/s06.dat  
  inflating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/s07.dat  
  inflating: datasets/DEAP/data_preprocessed_python/data_preprocessed_python/s08.dat  
  inflating: datasets/DEAP/data_preprocessed

In [ ]:

import torch
import os
from PIL import Image
from torch.utils import data
import numpy as np
import pandas as pd
from torchvision import transforms as T
import io
import zipfile


In [ ]:
# !unzip "/content/gdrive/Shareddrives/DEAP dataset/face_video.zip"

In [ ]:
class DEAP(data.Dataset):
    '''
    DEAP dataset for per-subject experiments
    Parameters:
        modal   : data modality; {'face', 'eeg', 'peri', 'bio', 'faceeeg', 'faceperi', 'facebio'}, default = 'facebio'.
        subject : subject ID; an integer between 1 and 22, default = 1.
        k       : kth fold; an integer between 1 and 10, default = 1.
        kind    : dataset type; {'train', 'val', 'all'}, default = 'all'.
        indices : index of data samples (for dataset shuffle); an list of integers, default = list(range(2400)).
        label   : emotion label; {'valence', 'arousal'}, defualt = 'valence'.
    '''
    def __init__(self, modal='facebio', subject=1, k=1, kind='all', indices=list(range(2400)),label='valence'):
        self.modal = modal
        self.subject = subject
        self.k = k
        self.kind = kind
        self.label = label
        self.bio_path = f'./data/DEAP/bio/s{subject}.zip'
        self.label_path = f'./data/DEAP/labels/'
        self.face_path = f'./data/DEAP/faces/s{subject}.zip'
        self.labels = pd.read_csv(self.label_path+'participant_ratings.csv')
        self.face_zip = zipfile.ZipFile(self.face_path, 'r')
        self.bio_zip = zipfile.ZipFile(self.bio_path, 'r')
        self.size = len(indices)

        if kind == 'train':
            self.indices = indices[:int((k - 1) * self.size / 10)] + indices[int(k * self.size / 10):]
        if kind == 'val':
            self.indices = indices[int((k - 1) * self.size / 10):int(k * self.size / 10)]
        if kind == 'all':
            self.indices = indices

    def __getitem__(self, i):
        index = self.indices[i]
        trial = index // 60 + 1
        segment = index % 60 + 1
        prex = 's' + (str(self.subject) if self.subject > 9 else '0' + str(self.subject)) + '/s' + (
            str(self.subject) if self.subject > 9 else '0' + str(self.subject)) + '_trial' + (
                   str(trial) if trial > 9 else '0' + str(trial)) + '/s' + (
                   str(self.subject) if self.subject > 9 else '0' + str(self.subject)) + '_trial' + (
                   str(trial) if trial > 9 else '0' + str(trial))
        transform = T.Compose([T.Resize((64, 64)),
                               T.ToTensor()])

        face_data = []
        for n in range(1, 6):
            img = Image.open(io.BytesIO(self.face_zip.read(prex + f'_{(segment - 1) * 5 + n}.png')))
            frame_array = transform(img)
            frame_array = frame_array.view(1, 3, 64, 64)
            face_data.append(frame_array)
        face_data = torch.cat(face_data, dim=0)
        bio_data = torch.tensor(
            np.load(io.BytesIO(self.bio_zip.read(f's{self.subject}/{self.subject}_{trial}_{segment}.npy')))).float()

        if self.modal == 'face':
            data = face_data
        elif self.modal == 'eeg':
            data = bio_data[:32]
        elif self.modal == 'peri':
            data = bio_data[32:]
        elif self.modal == 'bio':
            data = bio_data
        elif self.modal == 'faceeeg':
            data = (face_data, bio_data[:32])
        elif self.modal == 'faceperi':
            data = (face_data, bio_data[32:])
        elif self.modal == 'facebio':
            data = (face_data, bio_data)

        valence = 0 if self.labels[(self.labels['Participant_id']==self.subject) & (self.labels['Trial']==trial)]['Valence'].iloc[0] < 5 else 1
        arousal = 0 if self.labels[(self.labels['Participant_id']==self.subject) & (self.labels['Trial']==trial)]['Arousal'].iloc[0] < 5 else 1
        if self.label == 'valence':
            return data, valence
        else:
            return data, arousal

    def __len__(self):
        return len(self.indices)


In [ ]:
class DEAPAll(data.Dataset):
    def __init__(self, modal='facebio', k=1, kind='all', indices=list(range(52440)),label='valence'):
        self.modal = modal
        self.k = k
        self.kind = kind
        self.label = label
        self.bio_path = f'./data/DEAP/bio/'
        self.label_path = f'./data/DEAP/labels/'
        self.face_path = f'./data/DEAP/faces/'
        self.labels = pd.read_csv(self.label_path+'participant_ratings.csv')
        deap_indices_dict = {1: 2400,
                             2: 2400,
                             3: 2340,
                             4: 2400,
                             5: 2340,
                             6: 2400,
                             7: 2400,
                             8: 2400,
                             9: 2400,
                             10: 2400,
                             11: 2220,
                             12: 2400,
                             13: 2400,
                             14: 2340,
                             15: 2400,
                             16: 2400,
                             17: 2400,
                             18: 2400,
                             19: 2400,
                             20: 2400,
                             21: 2400,
                             22: 2400}
        self.sub_trial_seg = []
        for sub in range(1,23):
            for trial in range(1,int(deap_indices_dict[sub]/60+1)):
                for seg in range(1,61):
                    self.sub_trial_seg.append((sub,trial,seg))
        self.size = len(indices)

        if kind == 'train':
            self.indices = indices[:int((k - 1) * self.size / 10)] + indices[int(k * self.size / 10):]
        if kind == 'val':
            self.indices = indices[int((k - 1) * self.size / 10):int(k * self.size / 10)]
        if kind == 'all':
            self.indices = indices

    def __getitem__(self, i):
        index = self.indices[i]
        subject, trial, segment = self.sub_trial_seg[index]
        face_zip = zipfile.ZipFile(self.face_path+f's{subject}.zip', 'r')
        bio_zip = zipfile.ZipFile(self.bio_path+f's{subject}.zip', 'r')
        prex = 's' + (str(subject) if subject > 9 else '0' + str(subject)) + '/s' + (
            str(subject) if subject > 9 else '0' + str(subject)) + '_trial' + (
                   str(trial) if trial > 9 else '0' + str(trial)) + '/s' + (
                   str(subject) if subject > 9 else '0' + str(subject)) + '_trial' + (
                   str(trial) if trial > 9 else '0' + str(trial))
        transform = T.Compose([T.Resize((64, 64)),
                               T.ToTensor()])
        face_data = []
        for n in range(1, 6):
            img = Image.open(io.BytesIO(face_zip.read(prex + f'_{(segment - 1) * 5 + n}.png')))
            frame_array = transform(img)
            frame_array = frame_array.view(1, 3, 64, 64)
            face_data.append(frame_array)
        face_data = torch.cat(face_data, dim=0)

        bio_data = torch.tensor(np.load(io.BytesIO(bio_zip.read(f's{subject}/{subject}_{trial}_{segment}.npy')))).float()

        if self.modal == 'face':
            data = face_data
        elif self.modal == 'eeg':
            data = bio_data[:32]
        elif self.modal == 'peri':
            data = bio_data[32:]
        elif self.modal == 'bio':
            data = bio_data
        elif self.modal == 'faceeeg':
            data = (face_data, bio_data[:32])
        elif self.modal == 'faceperi':
            data = (face_data, bio_data[32:])
        elif self.modal == 'facebio':
            data = (face_data, bio_data)

        valence = 0 if self.labels[(self.labels['Participant_id']==subject) & (self.labels['Trial']==trial)]['Valence'].iloc[0] < 5 else 1
        arousal = 0 if self.labels[(self.labels['Participant_id']==subject) & (self.labels['Trial']==trial)]['Arousal'].iloc[0] < 5 else 1
        if self.label == 'valence':
            return data, valence
        else:
            return data, arousal

    def __len__(self):
        return len(self.indices)

In [ ]:
!pip install face_alignment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for face-alignment: filename=face_alignment-1.3.5-py2.py3-none-any.whl size=28241 sha256=c0473e68eebb5e4d8474080039aa9a5d2a356a82a35784a661f08406b56c86c1
  Stored in directory: /root/.cache/pip/wheels/c9/ba/4d/2d368f55e5f929f9472da59e356fbdf1483f885de80a5bc620
Successfully built face-alignment


In [ ]:
!pip install pyedflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 5.0 MB/s 


In [ ]:
"""
Functions for data pre-process
"""

import cv2
import numpy as np
import pandas as pd
import math
from collections import defaultdict
from PIL import Image
import pickle as cPickle
import os
import face_alignment
from xml.dom.minidom import parse
import pyedflib
import shutil

In [ ]:

# ************************* Face Data Pre-process *************************

def video2frames(dataset='DEAP'):
    '''
    Extract frames from videos.
    :param dataset: used dataset
    '''
    # assert dataset in ['DEAP', 'MAHNOB'], 'Invalid dataset name'
    assert dataset in ['DEAP'], 'Invalid dataset name'

    if dataset == 'DEAP':
        dataset_path = './datasets/DEAP/face_video/'
        des_path = './datasets/DEAP/frames/'
        for subject in os.listdir(dataset_path):
            if subject.startswith('.'):
                continue
            sub_path = dataset_path+subject
            for video_file in os.listdir(sub_path):
                if not os.path.exists(des_path + subject):
                    os.mkdir(des_path + subject)
                if not os.path.exists(des_path + subject + '/' + video_file.split('.')[0]):
                    os.mkdir(des_path + subject + '/' + video_file.split('.')[0])
                video_file_path = sub_path+'/'+video_file
                video = cv2.VideoCapture(video_file_path)
                c = 1
                frame_rate = 10
                count = 0
                while (True):
                    ret, frame = video.read()
                    if ret:
                        if (c % frame_rate == 0):
                            count += 1
                            cv2.imwrite(des_path+subject+'/'+video_file.split('.')[0] +'/'+ video_file.split('.')[0]+'_'+str(count) + '.png', frame)
                        c += 1
                        cv2.waitKey(0)
                    else:
                        break
                video.release()


In [ ]:
# functions for face alignment and cropping are based on https://github.com/DANNALI35/zhihu_article/tree/master/201901_face_alignment
def to_dict(landmarks):
    '''
    Transfer detected facial landmarks list to dictionary.
    :param landmarks: a list of facial landmarks
    :return: a dictionary of facial landmarks
    '''
    l = list()
    for i in range(68):
        point = (landmarks[i][0], landmarks[i][1])
        l.append(point)
    face_landmarks_dict = dict()
    face_landmarks_dict['chin'] = l[0:17]
    face_landmarks_dict['left_eyebrow'] = l[17:22]
    face_landmarks_dict['right_eyebrow'] = l[22:27]
    face_landmarks_dict['nose_bridge'] = l[27:31]
    face_landmarks_dict['nose_tip'] = l[31:36]
    face_landmarks_dict['left_eye'] = l[36:42]
    face_landmarks_dict['right_eye'] = l[42:48]
    face_landmarks_dict['top_lip'] = l[48:55] + l[60:65]
    face_landmarks_dict['bottom_lip'] = l[55:60] + l[65:68]
    return face_landmarks_dict


def crop_face(image_array, landmarks):
    """ crop face according to eye,mouth and chin position
    :param image_array: numpy array of a single image
    :param landmarks: dict of landmarks for facial parts as keys and tuple of coordinates as values
    :return:
    cropped_img: numpy array of cropped image
    """

    eye_landmark = np.concatenate([np.array(landmarks['left_eye']),
                                   np.array(landmarks['right_eye'])])
    eye_center = np.mean(eye_landmark, axis=0).astype("int")
    lip_landmark = np.concatenate([np.array(landmarks['top_lip']),
                                   np.array(landmarks['bottom_lip'])])
    lip_center = np.mean(lip_landmark, axis=0).astype("int")
    mid_part = lip_center[1] - eye_center[1]
    top = eye_center[1] - mid_part * 18 / 40
    bottom = lip_center[1] + mid_part * 12 / 40

    w = h = bottom - top
    x_center = eye_center[0]
    left, right = (x_center - w / 2, x_center + w / 2)

    pil_img = Image.fromarray(image_array)
    left, top, right, bottom = [int(i) for i in [left, top, right, bottom]]
    cropped_img = pil_img.crop((left, top, right, bottom))
    cropped_img = np.array(cropped_img)
    return cropped_img, left, top

In [ ]:
def rotate_landmarks(landmarks, eye_center, angle, row):
    """ rotate landmarks to fit the aligned face
    :param landmarks: dict of landmarks for facial parts as keys and tuple of coordinates as values
    :param eye_center: tuple of coordinates for eye center
    :param angle: degrees of rotation
    :param row: row size of the image
    :return: rotated_landmarks with the same structure with landmarks, but different values
    """
    rotated_landmarks = defaultdict(list)
    for facial_feature in landmarks.keys():
        for landmark in landmarks[facial_feature]:
            rotated_landmark = rotate(origin=eye_center, point=landmark, angle=angle, row=row)
            rotated_landmarks[facial_feature].append(rotated_landmark)
    return rotated_landmarks

In [ ]:
def rotate(origin, point, angle, row):
    """ rotate coordinates in image coordinate system
    :param origin: tuple of coordinates,the rotation center
    :param point: tuple of coordinates, points to rotate
    :param angle: degrees of rotation
    :param row: row size of the image
    :return: rotated coordinates of point
    """
    x1, y1 = point
    x2, y2 = origin
    y1 = row - y1
    y2 = row - y2
    angle = math.radians(angle)
    x = x2 + math.cos(angle) * (x1 - x2) - math.sin(angle) * (y1 - y2)
    y = y2 + math.sin(angle) * (x1 - x2) + math.cos(angle) * (y1 - y2)
    y = row - y
    return int(x), int(y)

In [ ]:
def align_face(image_array, landmarks):
    """ align faces according to eyes position
    :param image_array: numpy array of a single image
    :param landmarks: dict of landmarks for facial parts as keys and tuple of coordinates as values
    :return:
    rotated_img:  numpy array of aligned image
    eye_center: tuple of coordinates for eye center
    angle: degrees of rotation
    """
    # get list landmarks of left and right eye
    left_eye = landmarks['left_eye']
    right_eye = landmarks['right_eye']
    # calculate the mean point of landmarks of left and right eye
    left_eye_center = np.mean(left_eye, axis=0).astype("int")
    right_eye_center = np.mean(right_eye, axis=0).astype("int")
    # compute the angle between the eye centroids
    dy = right_eye_center[1] - left_eye_center[1]
    dx = right_eye_center[0] - left_eye_center[0]
    # compute angle between the line of 2 centeroids and the horizontal line
    angle = math.atan2(dy, dx) * 180. / math.pi
    # calculate the center of 2 eyes
    eye_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
                  (left_eye_center[1] + right_eye_center[1]) // 2)
    # at the eye_center, rotate the image by the angle
    rotate_matrix = cv2.getRotationMatrix2D(eye_center, angle, scale=1)
    rotated_img = cv2.warpAffine(image_array, rotate_matrix, (image_array.shape[1], image_array.shape[0]))
    return rotated_img, eye_center, angle

In [ ]:
def align_landmarks(landmarks):
    left_eye = landmarks['left_eye']
    right_eye = landmarks['right_eye']
    # calculate the mean point of landmarks of left and right eye
    left_eye_center = np.mean(left_eye, axis=0).astype("int")
    right_eye_center = np.mean(right_eye, axis=0).astype("int")
    # compute the angle between the eye centroids
    dy = right_eye_center[1] - left_eye_center[1]
    dx = right_eye_center[0] - left_eye_center[0]
    # compute angle between the line of 2 centeroids and the horizontal line
    angle = math.atan2(dy, dx) * 180. / math.pi
    # calculate the center of 2 eyes
    eye_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
                  (left_eye_center[1] + right_eye_center[1]) // 2)
#     rotated_landmarks = defaultdict(list)
    rotated_landmarks = []
    for facial_feature in landmarks.keys():
        for landmark in landmarks[facial_feature]:
            rotated_landmark = rotate(origin=eye_center, point=landmark, angle=angle, row=570)
#             rotated_landmarks[facial_feature].append(rotated_landmark)
            rotated_landmarks.append(rotated_landmark)
    return rotated_landmarks

In [ ]:
def face_detection_alignment_cropping(dataset='DEAP'):
    '''
    Transfer frames to faces by face detection, alignment and cropping.
    :param dataset: used dataset
    '''
    # assert dataset in ['DEAP', 'MAHNOB'], 'Invalid dataset name'
    assert dataset in ['DEAP'], 'Invalid dataset name'

    # facial landmarks detector; use gpu by changing device parameter to 'cuda', or do not use gpu by changing device parameter to 'cpu'
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device='cpu')

    if dataset == 'DEAP':
        root = './datasets/DEAP/frames/'
        des_path = './data/DEAP/faces/'

    # if dataset == 'MAHNOB':
    #     root = './datasets/MAHNOB/frames/'
    #     des_path = './data/DEAP/faces/'

    for subject in os.listdir(root):
        for trial in os.listdir(root+subject):
            # if os.path.exists(des_path + subject + '/' + trial):
            #     continue
            os.mkdir(des_path + subject + '/' + trial)
            if os.path.exists(des_path + subject + '/' + trial):
                continue
            for frame in os.listdir(root+subject+'/'+trial):
                frame_path = root + subject + '/' + trial + '/' + frame
                img = cv2.imread(frame_path)
                preds = fa.get_landmarks(img)
                try:
                    landmarks_list = preds[0]
                    landmarks_dict = to_dict(landmarks_list)
                    aligned_face, eye_center, angle = align_face(image_array=img, landmarks=landmarks_dict)
                    rotated_landmarks = rotate_landmarks(landmarks=landmarks_dict, eye_center=eye_center, angle=angle,
                                                     row=img.shape[0])
                    cropped_img, left, top = crop_face(image_array=aligned_face, landmarks=rotated_landmarks)

                    cv2.imwrite(des_path + subject + '/' + trial + '/' + frame, cropped_img)
                except:
                    print(f'Fail to get the face image: {frame}')

In [ ]:
# ************************* Bio-sensing Data Pre-process *************************

def trial2segments(dataset='DEAP'):
    '''
    Divide bio-sensing data of each trial to 1-second length segments, and perform baseline removal.
    Note, when dealing with MAHNOB-HCI dataset, EEG data should be common reference averaged, bandpass filtered and artefact removed using EEGLab,
    and preprocessed EEG data files (one file per trial) should be stored in './datasets/MAHNOB/eeg_preprocessed/ folder in .npy format.
    :param dataset: used dataset
    '''
    # assert dataset in ['DEAP', 'MAHNOB'], 'Invalid dataset name'
    assert dataset in ['DEAP'], 'Invalid dataset name'

    if dataset == 'DEAP':
        root = './datasets/DEAP/data_preprocessed_python/'
        des_path = './data/DEAP/bio/'
        labels = pd.read_csv('./data/DEAP/labels/participant_ratings.csv')
        for file in os.listdir(root):
            subject = file.split('.')[0]
            sub_id = int(subject[1:])
            os.mkdir(des_path + 's' + str(sub_id))
            f = open(root + file, 'rb')
            d = cPickle.load(f, encoding='latin1')
            data = d['data']
            for experiment in range(40):
                trial = labels[(labels['Participant_id'] == sub_id) & (labels['Experiment_id'] == experiment + 1)][
                    'Trial'].iloc[0]
                # baseline
                l = []
                for i in range(3):
                    l.append(data[experiment][:, i * 128:(i + 1) * 128])
                baseline_mean = sum(l) / 3
                # segments
                for i in range(60):
                    data_seg = data[experiment][:, 384 + i * 128:384 + (i + 1) * 128]
                    data_seg_removed = data_seg - baseline_mean
                    np.save(f'{des_path}s{sub_id}/{sub_id}_{trial}_{i + 1}.npy', data_seg_removed)

In [ ]:
def preprocess_demo():
    '''
    This function pre-processes DEAP dataset.
    Please unzip face_video.zip, data_preprocessed_python.zip and metadata_csv.zip from DEAP dataset in './datasets/DEAP/'.
    Then call this function, the preprocessed data will be stored in './data/DEAP/'.
    It is recommended to use a device with GPU, otherwise the face detection will be slow.
    Note that faces cannot be detected from some frames, these frames should be replaced with the neighbour frame manually.
    '''
    # pre-process face data
    if not os.path.exists('./datasets/DEAP/frames/'):
        os.mkdir('./datasets/DEAP/frames/')
    if not os.path.exists('./data/'):
        os.mkdir('./data/')
    if not os.path.exists('./data/DEAP/'):
        os.mkdir('./data/DEAP/')
    if not os.path.exists('./data/DEAP/faces/'):
        os.mkdir('./data/DEAP/faces/')
    if not os.path.exists('./data/DEAP/labels/'):
        os.mkdir('./data/DEAP/labels/')
    shutil.copy('./datasets/DEAP/metadata_csv/participant_ratings.csv', './data/DEAP/labels/participant_ratings.csv')
    video2frames('DEAP')
    face_detection_alignment_cropping('DEAP')

    # preprocess bio-sensing data
    if not os.path.exists('./data/DEAP/bio/'):
        os.mkdir('./data/DEAP/bio/')
    trial2segments('DEAP')


if __name__ == '__main__':
    preprocess_demo()

FileNotFoundError: ignored

### Neural network models in DeepVANet

In [ ]:
import torch
import torch.nn as nn
import time

In [ ]:
# The implementation of CONVLSTM are based on the code from
# https://github.com/ndrplz/ConvLSTM_pytorch/blob/master/convlstm.py
class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, padding):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: int
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=kernel_size,
                              padding=padding)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, height, width):
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))

In [ ]:
class ConvLSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size=3, padding=1):
        super(ConvLSTM, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.cell = ConvLSTMCell(input_dim=self.input_dim,
                                 hidden_dim=self.hidden_dim,kernel_size=kernel_size, padding=padding)

    def forward(self, input_tensor, time=None):

        b, _, _, h, w = input_tensor.size()

        hidden_state = self.cell.init_hidden(b,h,w)

        seq_len = input_tensor.size(1)

        h, c = hidden_state
        for t in range(seq_len):
            h, c = self.cell(input_tensor=input_tensor[:, t, :, :, :], cur_state=[h, c])
        return h

In [ ]:
class FaceFeatureExtractorCNN(nn.Module):
    def __init__(self):
        super(FaceFeatureExtractorCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 6 * 6, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1000, 2),
            nn.Sigmoid()
        )

    def forward(self,x):
        x = self.net(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def save(self, name=None):
        """
        save the model
        """
        if name is None:
            prefix = 'FaceFeatureExtractorCNN_'
            name = time.strftime(prefix + '%m%d_%H:%M:%S.pth')
        torch.save(self.state_dict(), name)
        return name

    def load(self, path):
        # self.load_state_dict(torch.load(path))
        self.load_state_dict(torch.load(path,map_location=torch.device('cpu')))


In [ ]:
class FaceFeatureExtractor(nn.Module):
    def __init__(self, feature_size=16, pretrain=True):
        super(FaceFeatureExtractor, self).__init__()
        cnn = FaceFeatureExtractorCNN()
        if pretrain:
            cnn.load('./pretrained_cnn.pth')
        self.cnn = cnn.net
        self.rnn = ConvLSTM(128, 128)
        self.fc = nn.Linear(128*6*6, feature_size)

    def forward(self, x):
        # input should be 5 dimension: (B, T, C, H, W)
        b, t, c, h, w = x.shape
        x = x.view(b * t, c, h, w)
        cnn_output = self.cnn(x)
        rnn_input = cnn_output.view(b, t, 128, 6, 6)
        rnn_output = self.rnn(rnn_input)
        rnn_output = torch.flatten(rnn_output, 1)
        output = self.fc(rnn_output)
        return output

In [ ]:
class BioFeatureExtractor(nn.Module):
    def __init__(self, input_size=32, feature_size=40):
        super(BioFeatureExtractor, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=24, kernel_size=5),
            nn.BatchNorm1d(num_features=24),
            nn.ReLU(inplace=True),
            nn.Conv1d(in_channels=24, out_channels=16, kernel_size=3),
            nn.BatchNorm1d(num_features=16),
            nn.ReLU(inplace=True),
            nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3),
            nn.BatchNorm1d(num_features=8),
            nn.ReLU(inplace=True),
        )
        self.fc = nn.Linear(8*120, feature_size)

    def forward(self,x):
        x = self.cnn(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        return x


In [ ]:
class DeepVANetVision(nn.Module):
    def __init__(self,feature_size=16,pretrain=True):
        super(DeepVANetVision,self).__init__()
        self.features = FaceFeatureExtractor(feature_size=feature_size,pretrain=pretrain)
        self.classifier = nn.Sequential(
            nn.Linear(feature_size, 20),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(20, 1),
            nn.Sigmoid()
        )

    def forward(self,x):
        x = self.features(x)
        x = self.classifier(x)
        x = x.squeeze(-1)
        return x

    def save(self, name=None):
        """
        save the model
        """
        if name is None:
            prefix = 'checkpoints/' + 'face_classifier_'
            name = time.strftime(prefix + '%m%d_%H:%M:%S.pth')
        torch.save(self.state_dict(), name)
        return name

    def load(self, path):
        self.load_state_dict(torch.load(path))
        # self.load_state_dict(torch.load(path,map_location=torch.device('cpu')))

In [ ]:
class DeepVANetBio(nn.Module):
    def __init__(self, input_size=32, feature_size=64):
        super(DeepVANetBio, self).__init__()
        self.features = BioFeatureExtractor(input_size=input_size, feature_size=feature_size)
        self.classifier = nn.Sequential(
            nn.Linear(feature_size, 20),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(20, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        x = x.squeeze(-1)
        return x

    def save(self, name=None):
        """
        save the model
        """
        if name is None:
            prefix = 'checkpoints/' + 'physiological_classifier_'
            name = time.strftime(prefix + '%m%d_%H:%M:%S.pth')
        torch.save(self.state_dict(), name)
        return name

    def load(self, path):
        self.load_state_dict(torch.load(path))
        # self.load_state_dict(torch.load(path, map_location=torch.device('cpu')))


In [ ]:
class DeepVANet(nn.Module):
    def __init__(self, bio_input_size=32, face_feature_size=16, bio_feature_size=64,pretrain=True):
        super(DeepVANet,self).__init__()
        self.face_feature_extractor = FaceFeatureExtractor(feature_size=face_feature_size,pretrain=pretrain)

        self.bio_feature_extractor = BioFeatureExtractor(input_size=bio_input_size, feature_size=bio_feature_size)

        self.classifier = nn.Sequential(
            nn.Linear(face_feature_size + bio_feature_size, 20),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(20, 1),
            nn.Sigmoid()
        )

    def forward(self,x):
        img_features = self.face_feature_extractor(x[0])
        bio_features = self.bio_feature_extractor(x[1])
        features = torch.cat([img_features,bio_features.float()],dim=1)
        output = self.classifier(features)
        output = output.squeeze(-1)
        return output

    def save(self, name=None):
        """
        save the model
        """
        if name is None:
            prefix = 'checkpoints/' + 'fusion_classifier_'
            name = time.strftime(prefix + '%m%d_%H:%M:%S.pth')
        torch.save(self.state_dict(), name)
        return name

    def load(self, path):
        self.load_state_dict(torch.load(path))
        # self.load_state_dict(torch.load(path,map_location=torch.device('cpu')))

### Util functions

In [ ]:
def out_put(string, verbose):
    '''
    Help function for verbose,
    output the string to destination path
    Parameters
    ----------
    string  :str,  the string to output
    verbose :str, the path to store the output
    '''
    with open(f"{verbose}.txt", "a") as f:
        f.write(string + "\n")

### Pretrained_cnn.pth

In [ ]:
# model = torch.load("/content/DeepVANet-main/pretrained_cnn.pth")
model = torch.load("/content/DeepVANet-main/pretrained_cnn.pth",map_location ='cpu')

### Train and test

In [ ]:
!pip install torchnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 676 kB 7.0 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
  Created wheel for torchnet: filename=torchnet-0.0.4-py3-none-any.whl size=29742 sha256=09bbb7169898ead4450048a87a2107fd9a9935b76c9065f28fe22a6fd84d7090
  Stored in directory: /root/.cache/pip/wheels/93/17/b3/86db1d93e9dae198813aa79831b403e4844d67986cf93894b5
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=93f1527b3d1d5c8876b5f281a8aceae6d74444e5b02e89b75a35fc0f4be67d57
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=641c1a0b0dd31015dde7064a1eed7e9f8860083d068d0d5962cfb456b5b413b8
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Succe

In [ ]:

from torchnet import meter
import torch
import os
from torch.utils.data import DataLoader
# from models import DeepVANetBio, DeepVANetVision, DeepVANet
# from dataset import DEAP, MAHNOB, DEAPAll, MAHNOBAll
# from utils import out_put

In [ ]:
def train(modal, dataset, subject, k, l, epoch, lr, batch_size, file_name, indices, face_feature_size=16, bio_feature_size=64, use_gpu=False, pretrain=True):
    '''
    Train and test the model. Output the results.
    :param modal: data modality
    :param dataset: used dataset
    :param subject: subject id
    :param k: kth fold
    :param l: emotional label
    :param epoch: the number of epoches
    :param lr: learn rate
    :param batch_size: training batach size
    :param file_name: result file name
    :param indices: a list of index of the dataset
    :param face_feature_size: face feature size
    :param bio_feature_size: bio-sensing feature size
    :param use_gpu: use gpu or not
    :param pretrain: use pretrained cnn nor not
    :return: the best test accuracy
    '''
    if use_gpu:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    directory = file_name.split('/')[-2]
    if not os.path.exists(f'./results/{dataset}/{modal}/'+directory):
        os.mkdir(f'./results/{dataset}/{modal}/'+directory)

    if dataset == 'DEAP':
        ############## inter-subjects ##############
        if subject == 0:
            train_data = DEAPAll(modal=modal, k=k, kind='train', indices=indices, label=l)
            val_data = DEAPAll(modal=modal, k=k, kind='val', indices=indices, label=l)
        ############## per-subjects ##############
        else:
            train_data = DEAP(modal=modal,subject=subject,k=k,kind='train',indices=indices, label=l)
            val_data = DEAP(modal=modal,subject=subject,k=k,kind='val',indices=indices, label=l)
        bio_input_size = 40
        peri_input_size = 8
    if dataset == 'MAHNOB':
        ############## inter-subjects  ##############
        if subject == 0:
            train_data = MAHNOBAll(modal=modal, k=k, kind='train', indices=indices, label=l)
            val_data = MAHNOBAll(modal=modal, k=k, kind='val', indices=indices, label=l)
        ############## per-subject #################
        else:
            train_data = MAHNOB(modal=modal,subject=subject,k=k,kind='train',indices=indices, label=l)
            val_data = MAHNOB(modal=modal,subject=subject,k=k,kind='val',indices=indices, label=l)
        bio_input_size = 38
        peri_input_size = 6

    # model
    if modal == 'face':
        model = DeepVANetVision(feature_size=face_feature_size,pretrain=pretrain).to(device)
    if modal == 'bio':
        model = DeepVANetBio(input_size=bio_input_size, feature_size=bio_feature_size).to(device)
    if modal == 'eeg':
        model = DeepVANetBio(input_size=32, feature_size=bio_feature_size).to(device)
    if modal == 'peri':
        model = DeepVANetBio(input_size=peri_input_size, feature_size=bio_feature_size).to(device)
    if modal == 'faceeeg':
        model = DeepVANet(bio_input_size=32, face_feature_size=face_feature_size, bio_feature_size=bio_feature_size, pretrain=pretrain).to(device)
    if modal == 'faceperi':
        model = DeepVANet(bio_input_size=peri_input_size, face_feature_size=face_feature_size, bio_feature_size=bio_feature_size, pretrain=pretrain).to(device)
    if modal == 'facebio':
        model = DeepVANet(bio_input_size=bio_input_size, face_feature_size=face_feature_size, bio_feature_size=bio_feature_size, pretrain=pretrain).to(device)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

    # criterion and optimizer
    criterion = torch.nn.BCELoss()
    lr = lr
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)

    # meters
    loss_meter = meter.AverageValueMeter()

    best_accuracy = 0
    best_epoch = 0

    # train
    for epoch in range(epoch):
        pred_label = []
        true_label = []

        loss_meter.reset()
        for ii, (data,label) in enumerate(train_loader):
            # print(ii)
            # train model
            if modal == 'faceeeg' or modal == 'faceperi' or modal == 'facebio':
                input = (data[0].float().to(device), data[1].float().to(device))
            else:
                input = data.float().to(device)
            label = label.float().to(device)

            optimizer.zero_grad()
            pred = model(input).float()
            # print(pred.shape,label.shape)
            loss = criterion(pred, label)
            loss.backward()
            optimizer.step()

            # meters update
            loss_meter.add(loss.item())

            pred = (pred >= 0.5).float().to(device).data
            pred_label.append(pred)
            true_label.append(label)

        pred_label = torch.cat(pred_label,0)
        true_label = torch.cat(true_label,0)

        train_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)
        out_put('Epoch: ' + 'train' + str(epoch) + '| train accuracy: ' + str(train_accuracy.item()), file_name)

        val_accuracy = val(modal, model, val_loader, use_gpu)

        out_put('Epoch: ' + 'train' + str(epoch) + '| train loss: ' + str(loss_meter.value()[0]) +
              '| val accuracy: ' + str(val_accuracy.item()), file_name)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_epoch = epoch
            model.save(f"{file_name}_best.pth")

    model.save(f'{file_name}.pth')

    perf = f"best accuracy is {best_accuracy} in epoch {best_epoch}" + "\n"
    out_put(perf,file_name)

    return best_accuracy

In [ ]:
@torch.no_grad()
def val(modal, model, dataloader, use_gpu):
    model.eval()
    if use_gpu:
      device = torch.device('cuda')
    else:
      device = torch.device('cpu')

    pred_label = []
    true_label = []

    for ii, (data, label) in enumerate(dataloader):
        if modal == 'faceeeg' or modal == 'faceperi' or modal == 'facebio':
            input = (data[0].float().to(device), data[1].float().to(device))
        else:
            input = data.float().to(device)
        label = label.to(device)
        pred = model(input).float()

        pred = (pred >= 0.5).float().to(device).data
        pred_label.append(pred)
        true_label.append(label)

    pred_label = torch.cat(pred_label, 0)
    true_label = torch.cat(true_label, 0)

    val_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)

    model.train()

    return 

### Decision-level fusion functions

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
# from utils import out_put
import os

In [ ]:
def adaboost(face_model_path, bio_model_path, bio_type, indices, dataset='DEAP', subject=1, k=1, label='valence', use_gpu=False):
    if use_gpu:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    # sub-classifiers
    face_model = DeepVANetVision().to(device)
    face_model.load(face_model_path)
    if dataset == 'DEAP':
        if bio_type == 'bio': bio_input_size = 40
        if bio_type == 'eeg': bio_input_size = 32
        if bio_type == 'peri': bio_input_size = 8
    if dataset == 'MAHNOB':
        if bio_type == 'bio': bio_input_size = 48
        if bio_type == 'eeg': bio_input_size = 32
        if bio_type == 'peri': bio_input_size = 6
    bio_model = DeepVANetBio(bio_input_size).to(device)
    bio_model.load(bio_model_path)

    if dataset == 'DEAP':
        face_train = DEAP(subject=subject, modal='face', label=label, kind='train', indices=indices, k=k)
        face_test = DEAP(subject=subject, modal='face', label=label, kind='val', indices=indices, k=k)
        bio_train = DEAP(subject=subject, modal=bio_type, label=label, kind='train', indices=indices, k=k)
        bio_test = DEAP(subject=subject, modal=bio_type, label=label, kind='val', indices=indices, k=k)
    if dataset == 'MAHNOB':
        face_train = MAHNOB(subject=subject, modal='face', label=label, kind='train', indices=indices, k=k)
        face_test = MAHNOB(subject=subject, modal='face', label=label, kind='val', indices=indices, k=k)
        bio_train = MAHNOB(subject=subject, modal=bio_type, label=label, kind='train', indices=indices, k=k)
        bio_test = MAHNOB(subject=subject, modal=bio_type, label=label, kind='val', indices=indices, k=k)

    n_train = len(face_train)
    n_test = len(face_test)

    face_train_loader = DataLoader(face_train, batch_size=64, shuffle=False)
    face_test_loader = DataLoader(face_test, batch_size=64, shuffle=False)
    bio_train_loader = DataLoader(bio_train, batch_size=64, shuffle=False)
    bio_test_loader = DataLoader(bio_test, batch_size=64, shuffle=False)

    # initialize weights
    w = np.ones(n_train) / n_train

    face_pred = []
    face_train_y = []
    for ii, (x, y) in enumerate(face_train_loader):
        print(ii)
        pred = face_model(x.to(device)).cpu().detach().numpy()
        face_pred.append(pred)
        face_train_y.append(y.detach().numpy())
    face_pred = np.concatenate(face_pred)
    face_train_y = np.concatenate(face_train_y)
    train_accuracy_face = sum((face_pred>=0.5).astype(float) == face_train_y) / n_train
    I = (face_pred==face_train_y).astype(float)
    I2 = np.array([1 if x==1 else -1 for x in I])
    error = sum(abs(face_pred-face_train_y)*w)
    alpha_face = 0.5 * np.log((1 - error)/error)
    w_updated = np.multiply(w, np.exp([float(x) * alpha_face for x in I2]))

    bio_pred = []
    bio_train_y = []
    for ii, (x, y) in enumerate(bio_train_loader):
        print(ii)
        pred = bio_model(x.to(device)).cpu().detach().numpy()
        bio_pred.append(pred)
        bio_train_y.append(y.detach().numpy())
    bio_pred = np.concatenate(bio_pred)
    bio_train_y = np.concatenate(bio_train_y)
    train_accuracy_bio = sum((bio_pred >= 0.5).astype(float) == bio_train_y) / n_train
    I = (bio_pred == bio_train_y).astype(float)
    I2 = np.array([1 if x == 1 else -1 for x in I])
    error = sum(abs(bio_pred - bio_train_y) * w_updated)
    alpha_bio = 0.5 * np.log((1 - error)/error)

    face_s = []
    face_test_y = []
    for ii, (x, y) in enumerate(face_test_loader):
        print(ii)
        pred = face_model(x.to(device)).cpu().detach().numpy()
        face_s.append(pred)
        face_test_y.append(y.detach().numpy())
    face_s = np.concatenate(face_s)
    face_test_y = np.concatenate(face_test_y)

    bio_s = []
    for ii, (x, y) in enumerate(bio_test_loader):
        print(ii)
        pred = bio_model(x.to(device)).cpu().detach().numpy()
        bio_s.append(pred)
    bio_s = np.concatenate(bio_s)

    face_s_mapped = 2 * face_s - 1
    bio_s_mapped = 2 * bio_s - 1

    final_score = 1 / (1 + np.exp(-(alpha_face*face_s_mapped+alpha_bio*bio_s_mapped)))

    final_pred = (final_score>=0.5).astype(float)
    test_accuracy = sum(final_pred==face_test_y) / n_test

    print(f'train accuracy face: {train_accuracy_face}, train accuracy bio: {train_accuracy_bio}, alpha_face: {alpha_face}, alpha_bio: {alpha_bio}, test accuracy: {test_accuracy}')
    return train_accuracy_face, train_accuracy_bio, alpha_face, alpha_bio, test_accuracy


In [ ]:
def decision_fusion(dataset, modal, subject, k, label, indices, use_gpu, pretrain=True):
    # train sub-classifiers
    bio_modal = modal[4:]
    if not os.path.exists(f'./results/'):
        os.mkdir(f'./results/')
    if not os.path.exists(f'./results/{dataset}/'):
        os.mkdir(f'./results/{dataset}/')
    if not os.path.exists(f'./results/{dataset}/face/'):
        os.mkdir(f'./results/{dataset}/face/')
    if not os.path.exists(f'./results/{dataset}/{bio_modal}/'):
        os.mkdir(f'./results/{dataset}/{bio_modal}/')
    train(modal='face', dataset=dataset, epoch=50, lr=0.001, use_gpu=use_gpu,
               file_name=f'./results/{dataset}/face/{dataset}_decision_f_{label}_s{subject}_k{k}/{dataset}_decision_f_{label}_s{subject}_k{k}',
               batch_size=64, subject=subject, k=k, l=label, indices=indices,pretrain=pretrain)
    train(modal=bio_modal, dataset=dataset, epoch=50, lr=0.001, use_gpu=use_gpu,
              file_name=f'./results/{dataset}/{bio_modal}/{dataset}_decision_{bio_modal}_{label}_s{subject}_k{k}/{dataset}_decision_{bio_modal}_{label}_s{subject}_k{k}',
              batch_size=64, subject=subject, k=k, l=label, indices=indices)
    face_model_path = f'./results/{dataset}/face/{dataset}_decision_f_{label}_s{subject}_k{k}/{dataset}_decision_f_{label}_s{subject}_k{k}.pth'
    bio_model_path = f'./results/{dataset}/{bio_modal}/{dataset}_decision_{bio_modal}_{label}_s{subject}_k{k}/{dataset}_decision_{bio_modal}_{label}_s{subject}_k{k}.pth'

    train_accuracy_face, train_accuracy_bio, alpha_face, alpha_bio, test_accuracy = adaboost(face_model_path, bio_model_path, bio_modal,indices, dataset, subject, k, label, use_gpu)
    out_put(f'train accuracy face: {train_accuracy_face}, train accuracy bio: {train_accuracy_bio}, alpha_face: {alpha_face}, alpha_bio: {alpha_bio}, test accuracy: {test_accuracy}', f'./results/{dataset}/{modal}/{dataset}_decision_{label}_s{subject}_k{k}_{modal}')





# Demo for per-subject experiment


In [ ]:
# import train
# from train import train
# from decision_level_fusion import decision_fusion
import random
import argparse
import os

In [ ]:
# deap subject_id: sample number
deap_indices_dict = {1: 2400,
             2: 2400,
             3: 2340,
             4: 2400,
             5: 2340,
             6: 2400,
             7: 2400,
             8: 2400,
             9: 2400,
             10: 2400,
             11: 2220,
             12: 2400,
             13: 2400,
             14: 2340,
             15: 2400,
             16: 2400,
             17: 2400,
             18: 2400,
             19: 2400,
             20: 2400,
             21: 2400,
             22: 2400,
             23: 2400,
             24: 2400,
             25: 2400,
             26: 2400,
             27: 2400,
             28: 2400,
             29: 2400,
             30: 2400,
             31: 2400,
             32: 2400}

In [ ]:
def demo():
    parser = argparse.ArgumentParser(description='Per-subject experiment')
    parser.add_argument('--dataset', '-d', default='DEAP', help='The dataset used for evaluation', type=str)
    parser.add_argument('--fusion', default='feature', help='Fusion strategy (feature or decision)', type=str)
    parser.add_argument('--epoch', '-e', default=50, help='The number of epochs in training', type=int)
    parser.add_argument('--batch_size', '-b', default=64, help='The batch size used in training', type=int)
    parser.add_argument('--learn_rate', '-l', default=0.001, help='Learn rate in training', type=float)
    parser.add_argument('--gpu', '-g', default='True', help='Use gpu or not', type=str)
    # parser.add_argument('--file', '-f', default='./results/results.txt', help='File name to save the results', type=str)
    parser.add_argument('--modal', '-m', default='facebio', help='Type of data to train', type=str)
    parser.add_argument('--subject', '-s', default=1, help='Subject id', type=int)
    parser.add_argument('--face_feature_size', default=16, help='Face feature size', type=int)
    parser.add_argument('--bio_feature_size', default=64, help='Bio feature size', type=int)
    parser.add_argument('--label', default='valence', help='Valence or arousal', type=str)
    parser.add_argument('--pretrain',default='True', help='Use pretrained CNN', type=str)
    parser.add_argument('-f')

    args = parser.parse_args()

    use_gpu = True if args.gpu == 'True' else False
    pretrain = True if args.pretrain == 'True' else False

    if args.dataset == 'DEAP':
        indices = list(range(deap_indices_dict[args.subject]))
    # if args.dataset == 'MAHNOB':
    #     indices = list(range(mahnob_indices_dict[args.subject]))
    # shuffle the dataset
    random.shuffle(indices)

    if not os.path.exists(f'./results/'):
        os.mkdir(f'./results/')
    if not os.path.exists(f'./results/{args.dataset}/'):
        os.mkdir(f'./results/{args.dataset}/')
    if not os.path.exists(f'./results/{args.dataset}/{args.modal}/'):
        os.mkdir(f'./results/{args.dataset}/{args.modal}/')

    for k in range(1, 11):
        if args.fusion == 'feature':
            train(modal=args.modal, dataset=args.dataset, epoch=args.epoch, lr=args.learn_rate, use_gpu=use_gpu,
                        file_name=f'./results/{args.dataset}/{args.modal}/{args.dataset}_{args.modal}_{args.label}_s{args.subject}_k{k}_{args.face_feature_size}_{args.bio_feature_size}/{args.dataset}_{args.modal}_{args.label}_s{args.subject}_k{k}_{args.face_feature_size}_{args.bio_feature_size}',
                        batch_size=args.batch_size, subject=args.subject, k=k, l=args.label, indices=indices,
                        face_feature_size=args.face_feature_size, bio_feature_size=args.bio_feature_size, pretrain=pretrain)
        if args.fusion == 'decision':
            decision_fusion(args.dataset, args.modal, args.subject, k, args.label, indices, use_gpu, pretrain)




In [ ]:
if __name__ == '__main__':
    demo()

FileNotFoundError: ignored

In [ ]:
%tb

SystemExit: ignored